DiracNets
=========

In this notebook we provide DiracNet-18-0.75 (12.8M parameters) model definitions with pretrained weights.
The model was trained using functional API of PyTorch on ILSVRC2012 train set.

top-1 and top-5 errors on ILSVRC2012 validation set:
**32.29, 12.16**

We saved the weights in hdf5 format, so that they can be loaded in other frameworks
without PyTorch.

In [143]:
import hickle as hkl
import torch
import torch.nn.functional as F
from torch.autograd import Variable

### load weights

In [144]:
params = hkl.load('./diracnet-18-0.75-export.hkl')

# convert numpy arrays to torch Variables
for k,v in sorted(params.items()):
    print k, v.shape
    params[k] = Variable(torch.from_numpy(v), requires_grad=True)
    
print '\nTotal parameters:', sum(v.numel() for v in params.values())

conv.weight (48, 3, 7, 7)
fc.bias (1000,)
fc.weight (1000, 384)
group0.block0.bn.a (48,)
group0.block0.bn.b (48,)
group0.block0.conv.bias (48,)
group0.block0.conv.weight (48, 96, 3, 3)
group0.block1.conv.bias (48,)
group0.block1.conv.weight (48, 96, 3, 3)
group0.block2.conv.bias (48,)
group0.block2.conv.weight (48, 96, 3, 3)
group0.block3.conv.weight (48, 96, 3, 3)
group1.block0.bn.a (48,)
group1.block0.bn.b (48,)
group1.block0.conv.bias (96,)
group1.block0.conv.weight (96, 96, 3, 3)
group1.block1.conv.bias (96,)
group1.block1.conv.weight (96, 192, 3, 3)
group1.block2.conv.bias (96,)
group1.block2.conv.weight (96, 192, 3, 3)
group1.block3.conv.weight (96, 192, 3, 3)
group2.block0.bn.a (96,)
group2.block0.bn.b (96,)
group2.block0.conv.bias (192,)
group2.block0.conv.weight (192, 192, 3, 3)
group2.block1.conv.bias (192,)
group2.block1.conv.weight (192, 384, 3, 3)
group2.block2.conv.bias (192,)
group2.block2.conv.weight (192, 384, 3, 3)
group2.block3.conv.weight (192, 384, 3, 3)
group3.blo

# Functional definition

In [145]:
def define_diracnet(depth):
    definitions = {18: [2,2,2,2], 34: [3,4,6,5]}
    blocks = definitions[depth]
    
    def ncrelu(x):
        return torch.cat([x.clamp(min=0), x.clamp(max=0)], dim=1)

    def group(o, params, base, count):
        for i in range(count):
            if i == 0:
                s = '%s.block%d.bn' % (base, i)
                a = params[s + '.a'].view(1,-1,1,1).expand_as(o)
                b = params[s + '.b'].view(1,-1,1,1).expand_as(o)
                o = o * a + b
            s = '%s.block%d.conv' % (base, i)
            weight = params[s + '.weight']
            bias = params[s + '.bias'] if i < count - 1 or base == 'group3' else None
            o = F.conv2d(ncrelu(o), weight=weight, bias=bias, padding=1)
        return o
    
    def f(inputs, params):
        o = F.conv2d(inputs, params['conv.weight'], padding=3, stride=2)
        o = F.max_pool2d(o, 3, 2, 1)
        o = group(o, params, 'group0', blocks[0] * 2)
        o = F.max_pool2d(o, 2)
        o = group(o, params, 'group1', blocks[1] * 2)
        o = F.max_pool2d(o, 2)
        o = group(o, params, 'group2', blocks[2] * 2)
        o = F.max_pool2d(o, 2)
        o = group(o, params, 'group3', blocks[3] * 2)
        o = F.avg_pool2d(F.relu(o), o.size(-1))
        o = o.view(o.size(0), -1)
        o = F.linear(o, params['fc.weight'], params['fc.bias'])
        return o
    
    return f

In [146]:
inputs = torch.randn(1,3,224,224)
y = define_diracnet(18)(Variable(inputs), params)
print y

Variable containing:
 0.5105  2.4716  1.5387  ...  -0.4935  0.3081  2.4002
[torch.FloatTensor of size 1x1000]



# Module definition

In [147]:
from torch import nn

# ugh modules are annoying

class NCReLU(nn.Module):
    def forward(self, x):
        return torch.cat([x.clamp(min=0), x.clamp(max=0)], dim=1)
    
    def __repr__(self):
        return 'NCReLU()'

class Affine(nn.Module):
    
    def __init__(self, channels):
        super(Affine, self).__init__()
        self.a = nn.Parameter(torch.Tensor(channels))
        self.b = nn.Parameter(torch.Tensor(channels))
        
    def forward(self, x):
        a = self.a.view(1,-1,1,1).expand_as(x)
        b = self.b.view(1,-1,1,1).expand_as(x)
        return x * a + b
    
    def __repr__(self):
        return 'Affine(%d)' % self.a.numel()
    
class Flatten(nn.Module):
    
    def forward(self, x):
        return x.view(x.size(0), -1)
    
    def __repr__(self):
        return 'Flatten()'

    
model = nn.Sequential()

model.add_module('conv', nn.Conv2d(in_channels=3, out_channels=48,
                                   kernel_size=7, stride=2, padding=3, bias=False))
model.add_module('max_pool0', nn.MaxPool2d(3, 2, 1))
model.add_module('group0.block0.bn', Affine(48))
for i in range(4):
    model.add_module('group0.block%d.ncrelu' % i, NCReLU())
    model.add_module('group0.block%d.conv' % i, nn.Conv2d(in_channels=96, out_channels=48,
                                                          kernel_size=3, padding=1, bias=i!=3))
model.add_module('max_pool1', nn.MaxPool2d(2))
model.add_module('group1.block0.bn', Affine(48))
for i in range(4):
    model.add_module('group1.block%d.ncrelu' % i, NCReLU())
    model.add_module('group1.block%d.conv' % i, nn.Conv2d(in_channels=96 if i==0 else 192, out_channels=96,
                                                          kernel_size=3, padding=1, bias=i!=3))
model.add_module('max_pool2', nn.MaxPool2d(2))
model.add_module('group2.block0.bn', Affine(96))
for i in range(4):
    model.add_module('group2.block%d.ncrelu' % i, NCReLU())
    model.add_module('group2.block%d.conv' % i, nn.Conv2d(in_channels=192 if i==0 else 384, out_channels=192,
                                                          kernel_size=3, padding=1, bias=i!=3))
model.add_module('max_pool3', nn.MaxPool2d(2))
model.add_module('group3.block0.bn', Affine(192))
for i in range(4):
    model.add_module('group3.block%d.ncrelu' % i, NCReLU())
    model.add_module('group3.block%d.conv' % i, nn.Conv2d(in_channels=384 if i==0 else 768, out_channels=384,
                                                          kernel_size=3, padding=1, bias=True))
model.add_module('relu', nn.ReLU())
model.add_module('avg_pool', nn.AvgPool2d(7))
model.add_module('view', Flatten())
model.add_module('fc', nn.Linear(in_features=384, out_features=1000))

In [148]:
model.load_state_dict({k: v.data for k,v in params.items()})

Check against functional model:

In [149]:
x = Variable(inputs)
(model(x) - define_diracnet(18)(x, params)).abs().sum().data[0]

0.0

In [150]:
print model

Sequential (
  (conv): Conv2d(3, 48, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (max_pool0): MaxPool2d (size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
  (group0.block0.bn): Affine(48)
  (group0.block0.ncrelu): NCReLU()
  (group0.block0.conv): Conv2d(96, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (group0.block1.ncrelu): NCReLU()
  (group0.block1.conv): Conv2d(96, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (group0.block2.ncrelu): NCReLU()
  (group0.block2.conv): Conv2d(96, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (group0.block3.ncrelu): NCReLU()
  (group0.block3.conv): Conv2d(96, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (max_pool1): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (group1.block0.bn): Affine(48)
  (group1.block0.ncrelu): NCReLU()
  (group1.block0.conv): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (group1.block1.ncrelu): NCReLU()
  (g